In [ ]:
# Uncomment if you are running the notebook on Google Colab.
# ! git clone https://github.com/jose-lopez/lemmas_finder.git

In [ ]:
import os

with os.popen("google-chrome --version") as f:
    browser = f.readlines()[0]

if not browser:

  try:

    print(os.popen('wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb').read())
    print(os.popen('apt install ./google-chrome-stable_current_amd64.deb').read())

  except Exception as exc:

      print(exc)

  else:

    with os.popen("google-chrome --version") as f:
        browser = f.readlines()[0]

print(f'Google version installed= {browser}')

In [ ]:
! pip install webdriver-manager

In [ ]:
! pip install selenium

In [ ]:
pip install pandas

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from urllib.parse import quote
from os import path
from pathlib import Path
import os
import pandas as pd
from numpy.core.numeric import nan
import re

In [ ]:
def get_browser():

    chrome_options = webdriver.ChromeOptions()

    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    chrome_options.add_argument("--headless=new")

    browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

    return browser

In [ ]:
def get_lemma(browser, file, line, token, logs):

    url_base = "https://logeion.uchicago.edu/morpho/"

    url = url_base + quote(token)

    browser.get(url)  # navigate to URL

    try:

        # Waiting for a totally deployed URL.

        WebDriverWait(browser, 10).until(EC.text_to_be_present_in_element((By.TAG_NAME, "h3"), "Short Definition"))

    except NoSuchElementException:

        lemma = nan

        logs.write(f'An exception of type NoSuchElementException in File: {file} at line: {line}, token {token}' + "\n")

    except TimeoutException:

        lemma = nan

        logs.write(f'An exception of type TimeoutException in File: {file} at line: {line}, token {token}' + "\n")

    else:

        lemma = browser.find_element(By.CSS_SELECTOR, 'a.ng-binding').text

        invalid_lemma = re.search(r'[a-zA-Z0-9]+', lemma)

        if invalid_lemma:

            lemma = nan

    finally:

        return lemma

In [ ]:
def check_warning(token, lemma):

    warning = None

    invalid_token = re.search(r'[a-zA-Z0-9]+', token)

    if lemma is not nan:

        invalid_lemma = re.search(r'[a-zA-Z0-9]+', lemma)

    else:

        invalid_lemma = None

    if invalid_token and not invalid_lemma:

        warning = 1

    if not invalid_token and invalid_lemma:

        warning = 2

    if invalid_token and invalid_lemma:

        warning = 3

    return warning

In [ ]:
! ls -l

In [ ]:
! pwd


In [ ]:
folders = ['processed', 'warnings', 'logs']

# Uncomment if you are running the notebook on Google Colab
root = "./lemmas_finder/text/"

# Uncomment if you are running the notebook your own Jupyter Kernel
root = "./text/"

corpus = root + "corpus"

for folder in folders:

  _path = root + folder
  if not path.exists(_path):
    os.mkdir(_path)

In [ ]:
# ls -l ./lemmas_finder/text/corpus
! ls -l ./text/corpus

In [ ]:
browser = get_browser()

In [ ]:
files = [str(x) for x in Path(corpus).glob("**/*.csv")]

files_to_process = len(files)

warnings_in_file = []

processed_files = 0

for file in files:

  file_name = "/" + file.split("/")[-1]

  processed_files += 1

  processed_file = root + folders[0] + file_name

  warnings_file = root + folders[1] + file_name

  logs_file = root + folders[2] + file_name

  logs = open(
      logs_file, 'w', encoding="utf8")

  input_df = pd.read_csv(file)

  print(f'Getting lemmas for {file} file: {processed_files} | {files_to_process}' + "\n")

  for x in input_df.index:

    token = input_df.loc[x, "token"]

    lemma = input_df.loc[x, "lemma"]

    warning = check_warning(token, lemma)

    if warning:

      warnings_in_file.append([x, token, lemma, warning])

    if lemma is nan:

      lemma = get_lemma(browser, file, x, token, logs)

      print(f'token = {token}   lemma = {lemma}' + "\n")

      input_df.loc[x, "lemma"] = lemma

  input_df.to_csv(processed_file)

  # Building the warnings' file, if there are any, for the actual file in process.

  if len(warnings_in_file) != 0:

    print(f'Errors found in {file} file. A report in {warnings_file}')

    warnings_df = pd.DataFrame(warnings_in_file, columns=['line', 'token', 'lemma', 'error_type'])

    warnings_df.to_csv(warnings_file)

  logs.close()

print(f'..... done')